## Setup 

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace
from IPython.display import Markdown as md


In [ ]:
import numpy as _np, numpy as np 
path_assets = './assets/'

## Graph building 

In [ ]:
def f(z):  return 1 / (1 + np.exp(-z))
def f2(z): return np.divide(1,np.add(1, np.exp(np.negative(z))))
def f3(z): 
    t1 = np.negative(z) 
    t2 = np.exp(t1)
    t3 = np.add(1, t2)
    y = np.divide(1,t3) 
    return y 

How to go from `f` to computation graph? 

Start with the easier problem: going from `f3` to computation graph. 

In [ ]:
# The basic structure to creating new functions 
def primitive(f): 
    def inner(*args, **kwargs): 
        print("add to graph!")
        return f(*args, **kwargs)
    return inner

# first way to use it 
new_mult = primitive(np.multiply)
print(new_mult(1,4))

# second way to use it 
@primitive 
def new_mult2(*args, **kwargs): return np.multiply(*args, **kwargs)
print(new_mult2(1,4))

add to graph!
4
add to graph!
4


In [ ]:
def primitive(fun): 
    """Apply this to differentiable functions"""
    def innerfun(*args, **kwargs):
        # code to seperate out the integer/non node case. sometimes you are adding 
        # constants to nodes. 
        def getval(o):      return o.value if type(o) == Node else o
        if len(args):       node_values = [getval(o) for o in args]
        if len(kwargs):     raise Exception("to implement")
        # print("add to graph!")
        return Node(fun(*node_values, **kwargs), fun, args)
    return innerfun

In [ ]:
def notrace_primitive(fun): 
    """Doesn't really do much. Evaluates functions, doesn't add them to the computation graph. """
    def innerfun(*args, **kwargs): 
        return fun(*args, **kwargs)
    return innerfun 

In [ ]:
# register some new functions 
# usually we'd do this for every function in numpy
add_new = primitive(np.add)
mul_new = primitive(np.multiply)
div_new = primitive(np.divide)
sub_new = primitive(np.subtract)
neg_new = primitive(np.negative)
exp_new = primitive(np.exp)


Here's a problem. When we wrap a function with this primitive stuff, we lose all the documentation of the original function. So `add_new` won't have any of the documentation of `np.add` in it. 

In [ ]:
print(np.multiply.__name__)
print(new_mult.__name__)

multiply
inner


What you want is some kind of "function machine", (i.e. a higher order function ) `f_copier` that takes in a function `f_no_docs`, makes the documentation of `f_no_docs` the same as another function `f_with_docs`, leaves `f_no_docs` unchanged otherwise, and then returns `f_no_docs`. Something like this. 

In [ ]:
def f_copier(f_with_docs): 
    def documentation_adder(f_no_docs): 
        f_no_docs.__doc__ = f_with_docs.__doc__
        f_no_docs.__name__ = f_with_docs.__name__
        return f_no_docs
    return documentation_adder

Higher order functions, or functions that return functions, can be used as decorators in Python. Any parameters you provide to the decorator get passed to the inner function, and if there is no inner function, you'll get an error. 

Let's look at how these work. Here is the no-parameter case:

In [ ]:
# changedoc is a function that changes documentation string of another function. 
def changedoc(f1): 
    f1.__doc__ = "a new doc"
    return f1

# Testing changedoc on some functions. 
# This one we use without a decorator. 
def somefun(x): return x+1
somefun = changedoc(somefun)
somefun.__doc__  # returns "a new doc"

@changedoc
def anotherfun(x): return x+2
anotherfun.__doc__  # returns "a new doc"

'a new doc'

Here we use decorators with parameters. Use the decorator with the outer function, and the parameter to the decorator is the same as the parameter to the outer function. Then the function under the decorator becomes the argument to the inner function. So the inner function must take a function as an argument. 

This one is a bit of a strange example. The decorator turns a function into an integer. 

In [ ]:
def add_x_to_f(x): 
    def inner(f): 
        """A function that adds the number x to the result of f() and returns the result"""
        print(locals())
        return f() + x 
    return inner

def twentyfour(): return 24
add_four = add_x_to_f(4)
add_four(twentyfour)

@add_x_to_f(3)
def nineteen(): return 16
nineteen

{'f': <function twentyfour at 0x10f69af28>, 'x': 4}
{'f': <function nineteen at 0x10531f488>, 'x': 3}


19

Here is another example. The decorator is called without arguments. This means `add_eighteen` becomes `f` in `add_x_to_f`. The function `add_eighteen` becomes `inner`, meaning it accepts a parameter, `x`. This is confusing because in the definition of `add_eighteen` it doesn't have any parameters, but the decorator gives it one. 

In [ ]:
def add_x_to_f(f): 
    def inner(x): 
        return f() + x
    return inner

@add_x_to_f
def add_eighteen(): return 18
add_eighteen(3) # gives 21

21

This works, but the decorator effectively doesn't do anything. There isn't any difference if we use the decorator or not. 

In [ ]:
def add_x_to_f(f): 
    def inner(*args): 
        return f(*args)
    return inner 

@add_x_to_f
def add_eighteen(x): return 18 + x
add_eighteen(4)

22

Here is a decorator used for timing functions. The function `add_stuff` goes into the `f` argument for `time_it`. The arguments `x` and `y` in `add_stuff` are caught by `*args` in `wrapper` and are accessed in a tuple in the `args` variable. 

In [ ]:
import time 
def time_it(f): 
    def inner(*args): 
        t = time.perf_counter()
        result = f(*args)
        print (f.__name__ + " takes " +  str(time.perf_counter() - t) + " seconds.")
        return result 
    return inner

@time_it
def add_stuff(x,y): return x+y
add_stuff(1,2)

add_stuff takes 2.776999735942809e-06 seconds.


3

Here is one for logging 

In [ ]:
def add_logs(f): 
    def inner(*args, **kwargs): 
        print('write', *args, "to file")
        return f(*args, **kwargs)
    return inner    

@add_logs
def somefun(x,y,z): return x+y+z
somefun(1,4,3)

This example runs some unit tests against a function the first time you define it. This is useful to see if you make a mistake or not. Also, if you change the definition of the function, the tests will run automatically. 

In [ ]:
def run_tests(tests): 
    def inner(f, *args, **kwargs):
        for test in tests: 
            params, result = test
            if f(*params) == result: print("Test", *params, "passed.")
            else:                    print("Test", *params, "failed.")
        return f
    return inner

@run_tests([
    [(1,2,4), False],
    [(1,2,7), True],
    [(10,0,0),True], 
    [(-10,10,10),True],
    [(4,0,7), False]
])
def adds_to_ten(x,y,z): return True if x+y+z==10 else False 

Test 1 2 4 passed.
Test 1 2 7 passed.
Test 10 0 0 passed.
Test -10 10 10 passed.
Test 4 0 7 passed.


We'll now convert this to the terminology used in autograd packages. Instead of `f_copier`, call it `wraps`, because eventually you can use this as a decorator to "wrap" a function. This inner function `documentation_adder` gets called `_wraps`. I'll keep the variable names 


In [ ]:
def wraps(f_docs)

Example: 
```
def f1(): 
"""docstring of f1"""
return 1

@wraps(f1) 
def f2(): return 2
```

which is the same as 
```
f_wraps = wraps(f1)
f2 = f_wraps(f1)
```
If you look at docstring and `__name__` of `f2`, they are the same as `f1`. 



In [ ]:
def wraps(f): 
    """
    Copies documentation and name of f2 to f. 
    Does this by returning a function that copies documentation of f to whatever you give it. 
    
    f is the function which you want to copy the documentation of. 
    f2 below is the new function that you want to have the same documentation as f. 
    """
    def inner(f2): 
        f2.__name__ = f.__name__
        f2.__doc__  = f.__doc__
        return f2
    return inner 

Here is how to use `wraps`. You can use it as a decorator or not as a decorator. It's easiest as a decorator. 

In [ ]:
### DECORATOR
# Copy documentation from np.add to some_add_fun
@wraps(np.add)
def some_add_fun(x,y): 
    return x+y

In [ ]:
### NON-DECORATOR 
def some_add_fun(x,y): 
    return x+y
np_add_wrapper = wraps(np.add)
some_add_fun =  np_add_wrapper(some_add_fun)
#some_add_fun now has the same documentation as np.add

In [ ]:
def primitive(f): 
    @wraps(f)
    def inner(*args, **kwargs): 
        print("add to graph!")
        return f(*args, **kwargs)
    return inner

In [ ]:
mul_new = primitive(np.multiply)
mul_new(5,2)

In [ ]:
def fun_with_docs(): """here are some textual things"""

In [ ]:
@wraps(fun_with_docs)
def a_fun(x): return x+1

In [ ]:
# f_that_adds_documentation = wraps(fun_with_docs)
# a_fun = f_that_adds_documentation(fun_with_docs)
# a_fun.__doc__


In [ ]:
class Node: 
    def __init__(self, value, fun,  parents): 
        self.value,self.fun,self.parents = value,fun,parents
        self.recipe = (fun, value)
        self.depth = 0 if parents == [] else 1+max(
            [o.depth if type(o)==Node else 0 for o in parents]) 
        
#     def get_max_depth(node, curr_max=0): 
#         if node.depth > curr_max: curr_max = node.depth 
#         for p in node.parents: print(p); get_max_depth(p, curr_max)
#         return curr_max
    
    def __repr__(self): 
        if self.value is None: str_val = 'None'
        else:                  str_val = str(round(self.recipe[1],3))
        return "\n" + self.depth * "\t" + "Fun: " + str(self.recipe[0]) +\
                " Value: "+ str_val + \
                " Parents: " + str(self.parents) 

    ## Overwrite operators to use our functions 
    # Don't put self.value or other.value in the arguments of these functions, 
    # otherwise you won't be able to access the Node object to create the 
    # computational graph. 
    # Instead, pass the whole node through. And to prevent recursion errors, 
    # extract the value inside the `primitive` function. 
    def __add__(self, other): return add_new(self, other)
    def __radd__(self, other): return add_new(other, self)
    def __sub__(self, other): return sub_new(self, other)
    def __rsub__(self, other): return sub_new(other, self)
    def __truediv__(self, other): return div_new(self, other)
    def __rtruediv__(self, other): return div_new(other, self)
    def __mul__(self, other): return mul_new(self, other)
    def __rmul__(self, other): return mul_new(other, self)
    def __neg__(self): return neg_new(self)
    def __exp__(self): return exp_new(self)
    


In [ ]:
# manual way with np functions 
val_z = 1.5 
z = Node(val_z, None, [])
val_t1 = np.negative(val_z)
t1 = Node(val_t1,np.negative, [z])
val_t2 = np.exp(val_t1)
t2 = Node(val_t2, np.exp, [t1])
val_t3 = np.add(val_t2, 1)
t3 = Node(val_t3, np.add, [t2])
val_y = np.divide(1, val_t3)
y = Node(val_y, np.divide, [t3])
print(y)

In [ ]:
# basic example of adding nodes 
val_z = 1.5 
z = Node(val_z, None, [])
val_t1 = 4
t1 = Node(val_t1, None, [])
z + t1 

Create a function to make the root node of the graph. 

In [ ]:
def new_root(value = None): 
    fun,parents = lambda x: x, []
    return Node(value, fun, parents)

In [ ]:
z = new_root(1.5)
t1 = -z
t2 = exp_new(t1)
t3 = t2 + 1
y = 1 / t3
y  # holds the graph

Now this works okay. But we are still specifying each intermediate variable. We want to get Python to build this graph for any function we like. 

In [ ]:
def f(z):  return 1 / (1 + exp_new(-z))

In [ ]:
y = f(new_root(value = 1.5))
y

Sweet! It is working. 

Now try for some multivariate functions. 

In [ ]:
def somefun(x,y):  return (x*y + exp_new(x)*exp_new(y))/(4*y)
def somefun2(x,y):  return (x*y + np.exp(x)*np.exp(y))/(4*y)

In [ ]:
val_x, val_y = 3,4 
somefun(new_root(3), new_root(4))

In [ ]:
somefun2(3,4)

The next bit of hackery is to get this working for all the functions in the numpy package. Then we can create our own version of numpy, import that as `np`, and don't have to worry about things like `exp_new`

Note that we will not want to add all functions in `numpy` to the computation graph if they are invoked. For example, stuff like `asarray` or things like that...don't add those. 

## Creating a new version of numpy 

In [ ]:
import numpy as _np
import types
import pandas as pd

In [ ]:
# numpy namespace
ns_numpy = _np.__dict__
ns_new = dict()

What's in the namespace of `numpy`? Let's look at the type of everything: 

In [ ]:
pd.Series([type(o) for o in ns_numpy.values()]).value_counts()

The main ones: `types.FunctionType`, `type`, `_np.ufunc`, `types.BuiltinFunctionType`, `int`, `types.ModuleType`, `float` and a bunch of others. 

The plan here:  

a) specify what functions have gradients and what don't  
b) create new functions by using `primitive` on functions with gradients, `notrace_primitive` on functions without. Save these new functions in the new namespace  
c) move everything else from old to new namespace   
d) deal with vjp's later  

In [ ]:
#triage system 

# functions with gradients

# functions without gradients 



In [ ]:
len(g1)
